In [ ]:
%run -i utilities.py

In [ ]:
# Set file paths and load files
file_path = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_bb_epoch.set')
mat_reject = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_reject_rmm.mat')
mat_stage = Path(
    'eeg-data/Stephanie/Rew_601_rest/Rew_601_rest_stages.mat')

In [ ]:
files = load_subject_dir(file_path, mat_reject, mat_stage)

In [ ]:
epochs = files['epochs']
try:
    reject = files['reject']
except:
    pass

try:
    stages = list(files['stages'])
except:
    pass

In [ ]:
index, scaling_time, scalings = ['epoch', 'time'], 1e3, dict(grad=1e13)
df_read = epochs.to_data_frame(
    picks=None, scalings=scalings, scaling_time=scaling_time, index=index)
df = clean_df(df_read)
df_ = df.copy()

In [ ]:
# Select values from dataframe
df_values = extract_df_values(df)
X = df_values.values
y = reject